In [ ]:
# Install required packages first
import subprocess
import sys

packages = ['tensorflow', 'scikeras']
for package in packages:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

print("✓ Packages installed successfully!\n")

# Now import all required libraries
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [ ]:
# Check if running on Google Colab and handle file upload
try:
    from google.colab import files
    COLAB = True
    print("✓ Running on Google Colab with GPU support!")
except ImportError:
    COLAB = False
    print("✓ Running locally")

In [ ]:
# For Google Colab: Upload CSV file
if COLAB:
    print("Upload your Churn_Modelling.csv file:")
    uploaded = files.upload()
    csv_file = list(uploaded.keys())[0]
    print(f"Uploaded: {csv_file}")
else:
    csv_file = 'Churn_Modelling.csv'
    print(f"Using local file: {csv_file}")

In [ ]:
# Load the data
data = pd.read_csv(csv_file)

data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

label_encode_gender=LabelEncoder()
data['Gender'] = label_encode_gender.fit_transform(data['Gender'])

one_hot_encoder_geo=OneHotEncoder(handle_unknown='ignore')
geo_encoder = one_hot_encoder_geo.fit_transform(data[['Geography']]).toarray()

geo_encoder_df=pd.DataFrame(geo_encoder,columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))

data=pd.concat([data.drop('Geography',axis=1),geo_encoder_df],axis=1)

X = data.drop('Exited', axis=1)
y = data['Exited']

print(f"Data loaded successfully! Shape: {data.shape}")

ModuleNotFoundError: No module named 'google.colab'

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.transform(X_test)

In [ ]:
# Save preprocessing objects as pickle files
with open('label_encode_gender.pkl', 'wb') as f:
    pickle.dump(label_encode_gender, f)
print("✓ Saved: label_encode_gender.pkl")

with open('one_hot_encoder_geo.pkl', 'wb') as f:
    pickle.dump(one_hot_encoder_geo, f)
print("✓ Saved: one_hot_encoder_geo.pkl")

with open('scalar.pkl', 'wb') as f:
    pickle.dump(scalar, f)
print("✓ Saved: scalar.pkl")

print("\nAll preprocessing objects saved successfully!")

# Download pickle files if on Colab
if COLAB:
    print("\nDownloading pickle files...")
    files.download('label_encode_gender.pkl')
    files.download('one_hot_encoder_geo.pkl')
    files.download('scalar.pkl')

✓ Saved: label_encode_gender.pkl
✓ Saved: one_hot_encoder_geo.pkl
✓ Saved: scalar.pkl

All preprocessing objects saved successfully!


In [30]:
# Defining a function to create model and try different parameters (KerasClassifier)

def create_model(neurons=32, layers=1):
    model = Sequential()
    model.add(Dense(neurons, activation='relu', input_shape=(X_train.shape[1],)))
    
    for _ in range(layers - 1):
        model.add(Dense(neurons, activation='relu'))
    
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [33]:
# Create KerasClassifier with the model builder function
model = KerasClassifier(model=create_model, epochs=50, batch_size=10, verbose=0)

In [ ]:
# Define hyperparameters to test
hyperparameters = [
    {'epochs': 50, 'batch_size': 10},
    {'epochs': 50, 'batch_size': 20},
    {'epochs': 100, 'batch_size': 10},
    {'epochs': 100, 'batch_size': 20},
]

print(f"Testing {len(hyperparameters)} configurations...\n")

In [ ]:
# Manual hyperparameter tuning (avoids scikeras/sklearn compatibility issues)
results = []
best_score = 0
best_params = None
best_model_obj = None

for i, params in enumerate(hyperparameters, 1):
    print(f"[{i}/{len(hyperparameters)}] Training with epochs={params['epochs']}, batch_size={params['batch_size']}")
    
    # Create fresh model for each configuration
    model = KerasClassifier(model=create_model, epochs=params['epochs'], 
                           batch_size=params['batch_size'], verbose=0)
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Evaluate on test set
    score = model.score(X_test, y_test)
    results.append({'params': params, 'score': score})
    
    print(f"   Test Accuracy: {score:.4f}\n")
    
    # Track best model
    if score > best_score:
        best_score = score
        best_params = params
        best_model_obj = model

# Print summary
print("="*50)
print("HYPERPARAMETER TUNING RESULTS")
print("="*50)
for r in results:
    print(f"Epochs: {r['params']['epochs']}, Batch Size: {r['params']['batch_size']} → Accuracy: {r['score']:.4f}")

print("\n" + "="*50)
print(f"Best Score: {best_score:.4f}")
print(f"Best Parameters: {best_params}")
print("="*50)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


KeyboardInterrupt: 

In [ ]:
# Save the best model
print("\nSaving the best model...")
best_model_obj.model_.save('churn_model_best.h5')
print("✓ Model saved as: churn_model_best.h5")

# Also save model summary
print("\nBest Model Summary:")
best_model_obj.model_.summary()

# Download model if on Colab
if COLAB:
    print("\nDownloading model...")
    files.download('churn_model_best.h5')